# 132: Stroke Progression + PQTL

Reminder, you can run something like:

`jupyter nbconvert --to html yourNotebook.ipynb`


# Manhattan Plots:

## First AIS (Existing Published GWAS vs. our meta-analysed GWAS)

![image](scratch/results/existing_stroke_gwas.png)
![image](scratch/results/meta_analysis/first_ais_metal_metaanalysis_manhattan.png)

Comparing by eye, they align quite well, with the existing stroke GWAS having much better results due to high power. HOWEVER, taking the top hitting 30 SNP results don't match  well with our results.  Therefore, I downloaded the full summary stats, and am looking at the wider range of hits.

## Subsequent GWASes (meta-analysed only)

![image](scratch/results/meta_analysis/subsequent_ais_metal_metaanalysis_manhattan.png)
![image](scratch/results/meta_analysis/subsequent_mace_metal_metaanalysis_manhattan.png)

## Subsequent GWASes (adjusted for collider bias, then meta-analysed)

add me later

In [27]:
source("gwas_data_helpers.R")
library(dplyr)
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
gwas_columns <- c("MARKER", "BETA", "SE", "P")

first_ais_metaanalysis <- data.table::fread("scratch/results/meta_analysis/first_ais_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_ais_metaanalysis_not_adjusted <- data.table::fread("scratch/results/meta_analysis/subsequent_ais_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_mace_metaanalysis_not_adjusted <- data.table::fread("scratch/results/meta_analysis/subsequent_mace_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_ais_metaanalysis_adjusted <- data.table::fread("scratch/results/subsequent_ais_metaanalysis_slopehunter1.txt", select = gwas_columns)
subsequent_mace_metaanalysis_adjusted <- data.table::fread("scratch/results/subsequent_mace_metaanalysis_slopehunter1.txt", select = gwas_columns)
subsequent_ais_metaanalysis_dudbridge <- data.table::fread("scratch/results/subsequent_ais_metaanalysis_dudbridge1.txt", select = gwas_columns)
subsequent_mace_metaanalysis_dudbridge <- data.table::fread("scratch/results/subsequent_mace_metaanalysis_dudbridge1.txt", select = gwas_columns)

existing_stroke_gwas <- data.table::fread("scratch/data/existing_stroke_gwas.tsv.gz")
existing_stroke_gwas <- existing_stroke_gwas %>% dplyr::rename("CHR" = "chromosome", "BP" = "base_pair_location", "P" = "p_value",
                                                               "BETA" = "beta", "OR" = "odds_ratio", "LB" = "ci_lower", "UB" = "ci_upper")
existing_stroke_gwas_top_hits <- existing_stroke_gwas[existing_stroke_gwas$P < 5e-8,]
existing_stroke_gwas_top_hits$MARKER <- paste(existing_stroke_gwas_top_hits$CHR, existing_stroke_gwas_top_hits$BP, sep=":")
existing_stroke_gwas_top_hits <- convert_odds_ratio_to_beta_estimate(existing_stroke_gwas_top_hits)

merge_gwases <- function(gwas) {
    merged_stroke_data_comparison <- merge(gwas, subsequent_ais_metaanalysis_not_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("EXISTING_BETA" = "BETA.x", "EXISTING_P" = "P.x", "EXISTING_SE" = "SE.x",
                                                                                     "SUB_AIS_BETA" = "BETA.y", "SUB_AIS_P" = "P.y", "SUB_AIS_SE" = "SE.y")

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_ais_metaanalysis_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_AIS_SH_BETA" = "BETA", "SUB_AIS_SH_P" = "P", "SUB_AIS_SH_SE" = "SE")
    
    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_ais_metaanalysis_dudbridge, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_AIS_DB_BETA" = "BETA", "SUB_AIS_DB_P" = "P", "SUB_AIS_DB_SE" = "SE")    

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_not_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_BETA" = "BETA", "SUB_MACE_P" = "P", "SUB_MACE_SE" = "SE")

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_SH_BETA" = "BETA", "SUB_MACE_SH_P" = "P", "SUB_MACE_SH_SE" = "SE")
    
    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_dudbridge, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_DB_BETA" = "BETA", "SUB_MACE_DB_P" = "P", "SUB_MACE_DB_SE" = "SE")

    return(merged_stroke_data_comparison)
}

merged_existing_gwas <- merge_gwases(existing_stroke_gwas_top_hits)

## Comparing p-values across all the different GWASes

There are two interesting things to notice from this table:
* In general, the subsequent MACE hits seem to align more closely with the incidence than the subsequent AIS does.
* SlopeHunter correction decimates the p-values of the MACE hits that show promise.



In [28]:
merged_existing_gwas <- merge_gwases(existing_stroke_gwas_top_hits)
merged_existing_gwas <- merge(merged_existing_gwas, first_ais_metaanalysis, by = "MARKER")
merged_existing_gwas <- merged_existing_gwas %>% dplyr::rename("FIRST_BETA" = "BETA", "FIRST_P" = "P", "FIRST_SE" = "SE")

merged_existing_gwas <- transform(merged_existing_gwas, SUB_AIS_MIN_P_VALUE = pmin(SUB_AIS_P, SUB_AIS_SH_P, SUB_AIS_DB_P))
merged_existing_gwas <- transform(merged_existing_gwas, SUB_MACE_MIN_P_VALUE = pmin(SUB_MACE_P, SUB_MACE_SH_P, SUB_MACE_DB_P))
print("Subsequent AIS Top Hits (when filtered by existing incidence GWAS 5e-8)")
head(merged_existing_gwas[with(merged_existing_gwas, order(SUB_AIS_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")], n=50)
print("Subsequent MACE Top Hits (when filtered by existing incidence GWAS 5e-8)")
head(merged_existing_gwas[with(merged_existing_gwas, order(SUB_MACE_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")], n=50)

[1] "Subsequent AIS Top Hits (when filtered by existing incidence GWAS 5e-8)"


MARKER,EXISTING_P,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7:92243719,3.114e-09,3.784e-01,0.0005098,0.0004147,0.0002142
4:155524009,1.972e-10,8.933e-02,0.0002233,0.0030280,0.0016460
5:121518309,5.642e-11,5.345e-01,0.0002285,0.0536200,0.0542800
6:1364471,5.152e-09,3.910e-04,0.0002929,0.0292000,0.0046930
4:155537601,1.792e-09,1.002e-01,0.0003129,0.0029890,0.0017960
4:155525970,4.293e-10,8.632e-02,0.0003371,0.0039690,0.0020330
5:121522778,4.083e-09,5.504e-01,0.0003607,0.0550000,0.0443700
6:1364691,1.695e-11,1.611e-05,0.0003946,0.0563700,0.0063920
9:22124450,1.167e-11,1.181e-05,0.0004771,0.0105600,0.0005860


[1] "Subsequent MACE Top Hits (when filtered by existing incidence GWAS 5e-8)"


MARKER,EXISTING_P,FIRST_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9:22114495,1.400e-09,1.170e-04,2.360e-06,0.0003476,2.501e-05
9:22124450,1.167e-11,1.181e-05,3.775e-06,0.0036250,2.648e-04
9:22124630,1.600e-11,1.063e-05,3.887e-06,0.0037080,2.741e-04
9:22088094,3.831e-13,7.124e-06,1.506e-05,0.0059720,4.051e-04
9:22115026,6.147e-12,1.093e-04,4.201e-04,0.0002760,2.065e-05
9:22115286,1.015e-13,1.019e-04,3.444e-04,0.0003231,2.672e-05
9:22112599,3.713e-13,9.049e-05,2.693e-05,0.0100000,9.315e-04
9:22116220,2.739e-13,1.013e-04,3.446e-04,0.0003384,2.806e-05
9:22088260,1.085e-08,7.287e-06,2.950e-05,0.0037600,2.424e-04


In [11]:
significant_first_ais_comparison <- first_ais_metaanalysis[first_ais_metaanalysis$P < 5e-8,]

significant_first_ais_comparison <- merge(significant_first_ais_comparison, subsequent_ais_metaanalysis_not_adjusted, by.x = "MARKER", by.y = "MARKER")
significant_first_ais_comparison <- significant_first_ais_comparison %>% dplyr::rename("FIRST_BETA" = "BETA.x", "FIRST_P" = "P.x", "FIRST_SE" = "SE.x",
                                                                                       "SUB_AIS_BETA" = "BETA.y", "SUB_AIS_P" = "P.y", "SUB_AIS_SE" = "SE.y")

significant_first_ais_comparison <- merge(significant_first_ais_comparison, subsequent_ais_metaanalysis_adjusted, by.x = "MARKER", by.y = "MARKER")
significant_first_ais_comparison <- significant_first_ais_comparison %>% dplyr::rename("SUB_AIS_SH_BETA" = "BETA", "SUB_AIS_SH_P" = "P", "SUB_AIS_SH_SE" = "SE")

significant_first_ais_comparison <- merge(significant_first_ais_comparison, subsequent_mace_metaanalysis_not_adjusted, by.x = "MARKER", by.y = "MARKER")
significant_first_ais_comparison <- significant_first_ais_comparison %>% dplyr::rename("SUB_MACE_BETA" = "BETA", "SUB_MACE_P" = "P", "SUB_MACE_SE" = "SE")

significant_first_ais_comparison <- merge(significant_first_ais_comparison, subsequent_mace_metaanalysis_adjusted, by.x = "MARKER", by.y = "MARKER")
significant_first_ais_comparison <- significant_first_ais_comparison %>% dplyr::rename("SUB_MACE_SH_BETA" = "BETA", "SUB_MACE_SH_P" = "P", "SUB_MACE_SH_SE" = "SE")

significant_first_ais_comparison[, c("MARKER", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_MACE_P", "SUB_MACE_SH_P")]

MARKER,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_MACE_P,SUB_MACE_SH_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10:105606084,1.362e-07,0.450200,0.0902700,2.189e-01,0.4450000
10:105615462,1.178e-07,0.442300,0.0692900,2.404e-01,0.4828000
10:105634589,1.876e-07,0.508700,0.1239000,2.618e-01,0.6427000
10:105660445,1.870e-07,0.389600,0.1022000,1.577e-01,0.3964000
10:105669289,2.127e-07,0.407800,0.0961300,1.490e-01,0.3867000
10:105669301,2.259e-07,0.398700,0.0884500,1.547e-01,0.4085000
10:105669304,2.259e-07,0.398700,0.0884500,1.547e-01,0.4085000
11:102994739,3.587e-07,0.299000,0.0640700,1.899e-01,0.0452500
12:111826477,2.318e-07,0.616900,0.7128000,2.353e-01,0.4545000


## Working Backwards from the Subsequent GWAS Results



In [15]:
significant_subsequent_ais_comparison <- subsequent_ais_metaanalysis_not_adjusted[subsequent_ais_metaanalysis_not_adjusted$P < 5e-6,]

significant_subsequent_ais_comparison <- merge(significant_subsequent_ais_comparison, first_ais_metaanalysis, by.x = "MARKER", by.y = "MARKER")
significant_subsequent_ais_comparison <- significant_subsequent_ais_comparison %>% dplyr::rename("FIRST_BETA" = "BETA.y", "FIRST_P" = "P.y", "FIRST_SE" = "SE.y",
                                                                                       "SUB_AIS_BETA" = "BETA.x", "SUB_AIS_P" = "P.x", "SUB_AIS_SE" = "SE.x")

significant_subsequent_ais_comparison <- merge(significant_subsequent_ais_comparison, subsequent_ais_metaanalysis_adjusted, by.x = "MARKER", by.y = "MARKER")
significant_subsequent_ais_comparison <- significant_subsequent_ais_comparison %>% dplyr::rename("SUB_AIS_SH_BETA" = "BETA", "SUB_AIS_SH_P" = "P", "SUB_AIS_SH_SE" = "SE")

significant_subsequent_ais_comparison[, c("MARKER", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P")]

significant_subsequent_mace_comparison <- subsequent_mace_metaanalysis_not_adjusted[subsequent_mace_metaanalysis_not_adjusted$P < 5e-6,]

significant_subsequent_mace_comparison <- merge(significant_subsequent_mace_comparison, first_ais_metaanalysis, by.x = "MARKER", by.y = "MARKER")
significant_subsequent_mace_comparison <- significant_subsequent_mace_comparison %>% dplyr::rename("FIRST_BETA" = "BETA.y", "FIRST_P" = "P.y", "FIRST_SE" = "SE.y",
                                                                                       "SUB_MACE_BETA" = "BETA.x", "SUB_MACE_P" = "P.x", "SUB_MACE_SE" = "SE.x")

significant_subsequent_mace_comparison <- merge(significant_subsequent_mace_comparison, subsequent_mace_metaanalysis_adjusted, by.x = "MARKER", by.y = "MARKER")
significant_subsequent_mace_comparison <- significant_subsequent_mace_comparison %>% dplyr::rename("SUB_MACE_SH_BETA" = "BETA", "SUB_MACE_SH_P" = "P", "SUB_MACE_SH_SE" = "SE")

significant_subsequent_mace_comparison[, c("MARKER", "FIRST_P", "SUB_MACE_P", "SUB_MACE_SH_P")]

MARKER,BETA,SE,P
<chr>,<dbl>,<dbl>,<dbl>
16:4417338,-0.1041,0.0219,1.957e-06
21:27395690,0.4827,0.0987,1.018e-06
17:73843221,0.3179,0.0630,4.578e-07
8:98869929,-0.1833,0.0371,7.589e-07
18:20832381,0.2715,0.0586,3.656e-06
14:73295026,-0.1024,0.0224,4.786e-06


MARKER,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P
<chr>,<dbl>,<dbl>,<dbl>
10:81137663,0.06431,3.447e-06,8.203e-05
10:81137997,0.05510,4.097e-06,8.315e-05
11:11298741,0.41080,3.640e-06,4.938e-06
11:75526346,0.65160,4.969e-06,6.868e-05
12:108665953,0.15780,4.861e-06,7.769e-05
12:108666175,0.15500,4.522e-06,7.310e-05
12:8340284,0.63370,3.674e-06,1.159e-01
14:73294871,0.17490,2.290e-06,3.783e-05
14:73295026,0.23150,4.786e-06,3.508e-04


MARKER,FIRST_P,SUB_MACE_P,SUB_MACE_SH_P
<chr>,<dbl>,<dbl>,<dbl>
11:75526346,6.516e-01,4.650e-06,3.313e-06
11:76311555,8.962e-01,4.138e-06,2.516e-05
11:76319155,9.317e-01,2.420e-06,1.542e-05
11:76322323,8.799e-01,2.470e-06,1.619e-05
12:8340284,6.337e-01,2.130e-06,7.484e-01
13:112784083,8.324e-01,1.007e-06,4.366e-01
13:37007584,2.388e-01,4.958e-06,2.342e-04
13:62088326,6.138e-02,4.924e-06,1.570e-03
15:32208140,2.500e-01,2.613e-06,1.500e-05
